# Advanced Evaluations

## Setup

To start, let's load our environment variables from our .env file. Make sure all of the keys necessary in .env.example are included!


In [35]:
from multiagent import multiagent

from dotenv import load_dotenv
load_dotenv(dotenv_path="../.env", override=True)

from langsmith import Client # Set up LangSmith client
client = Client()

from langchain_openai import ChatOpenAI
model = ChatOpenAI(model="o3-mini")

## Evaluations


To recap, Evaluations are made up of three components:

1. A **dataset test** inputs and expected outputs.
2. An **application or target function** that defines what you are evaluating, taking in inputs and returning the application output
3. **Evaluators** that score your target function's outputs.

![Evaluation](../images/evals-conceptual.png) 

There are many ways you can evaluate an agent. Today, we will cover the three common types of agent evaluations:

1. **Final Response**: Evaluate the agent's final response.
2. **Single step**: Evaluate any agent step in isolation (e.g., whether it selects the appropriate tool).
3. **Trajectory**: Evaluate whether the agent took the expected path (e.g., of tool calls) to arrive at the final answer.

### Evaluating The Final Response

One way to evaluate an agent is to assess its overall performance on a task. This basically involves treating the agent as a black box and simply evaluating whether or not it gets the job done.
- Input: User input 
- Output: The agent's final response.


![final-response](../images/final-response.png) 

#### 1. Create a Dataset

In [36]:
from langsmith import Client

client = Client()

# Create a dataset
examples = [
    {
        "question": "My name is Aaron Mitchell. My customer ID is 32. I am trying to find the invoice number for my most recent song purchase. Could you help me with it?",
        "response": "The Invoice ID of your most recent purchase was 342.",
    },
    {
        "question": "I'd like a refund.",
        "response": "I need additional information to help you with the refund. Could you please provide your customer identifier so that we can fetch your purchase history?",
    },
    {
        "question": "Who recorded Wish You Were Here again?",
        "response": "Wish You Were Here is an album by Pink Floyd",
    },
    { 
        "question": "What albums do you have by Coldplay?",
        "response": "There are no Coldplay albums available in our catalog at the moment.",
    },
]

dataset_name = "Evals: Final Response"

if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name=dataset_name)
    client.create_examples(
        inputs=[{"question": ex["question"]} for ex in examples],
        outputs=[{"response": ex["response"]} for ex in examples],
        dataset_id=dataset.id
    )

#### 2. Define Application Logic to be Evaluated 

Now, let's define how to run our graph. Note that here we must continue past the interrupt() by supplying a Command(resume="") to the graph.

In [37]:
import uuid
from langgraph.types import Command

graph = multiagent

async def run_graph(inputs: dict):
    """Run graph and track the final response."""
    # Creating configuration 
    thread_id = uuid.uuid4()
    configuration = {"thread_id": thread_id, "user_id" : "10"}

    # Invoke graph until interrupt 
    result = await graph.ainvoke({"messages": [
        { "role": "user", "content": inputs['question']}]}, config = configuration)
    # Proceed from human-in-the-loop 
    result = await graph.ainvoke(Command(resume="My customer ID is 10"), config={"thread_id": thread_id, "user_id" : "10"})
    
    return {"response": result['messages'][-1].content}

#### 3. Define the Evaluator

We can use pre-built evaluators from the `openevals` library

In [38]:
from openevals.llm import create_llm_as_judge
from openevals.prompts import CORRECTNESS_PROMPT

# Using Open Eval pre-built 
correctness_evaluator = create_llm_as_judge(
    prompt=CORRECTNESS_PROMPT,
    feedback_key="correctness",
    judge=model,
)
print(CORRECTNESS_PROMPT)

You are an expert data labeler evaluating model outputs for correctness. Your task is to assign a score based on the following rubric:

<Rubric>
  A correct answer:
  - Provides accurate and complete information
  - Contains no factual errors
  - Addresses all parts of the question
  - Is logically consistent
  - Uses precise and accurate terminology

  When scoring, you should penalize:
  - Factual errors or inaccuracies
  - Incomplete or partial answers
  - Misleading or ambiguous statements
  - Incorrect terminology
  - Logical inconsistencies
  - Missing key information
</Rubric>

<Instructions>
  - Carefully read the input and output
  - Check for factual accuracy and completeness
  - Focus on correctness of information rather than style or verbosity
</Instructions>

<Reminder>
  The goal is to evaluate factual correctness and completeness of the response.
</Reminder>

<input>
{inputs}
</input>

<output>
{outputs}
</output>

Use the reference outputs below to help you evaluate the

We can also define our own evaluator.

In [39]:
from typing_extensions import TypedDict
from typing import Annotated, List

# Custom definition of LLM-as-judge instructions
grader_instructions = """You are a teacher grading a quiz.

You will be given a QUESTION, the GROUND TRUTH (correct) RESPONSE, and the STUDENT RESPONSE.

Here is the grade criteria to follow:
(1) Grade the student responses based ONLY on their factual accuracy relative to the ground truth answer.
(2) Ensure that the student response does not contain any conflicting statements.
(3) It is OK if the student response contains more information than the ground truth response, as long as it is factually accurate relative to the ground truth response.

Correctness:
True means that the student's response meets all of the criteria.
False means that the student's response does not meet all of the criteria.

Explain your reasoning in a step-by-step manner to ensure your reasoning and conclusion are correct."""

# LLM-as-judge output schema
class Grade(TypedDict):
    """Compare the expected and actual answers and grade the actual answer."""
    reasoning: Annotated[str, ..., "Explain your reasoning for whether the actual response is correct or not."]
    is_correct: Annotated[bool, ..., "True if the student response is mostly or exactly correct, otherwise False."]

# Judge LLM
grader_llm = model.with_structured_output(Grade, method="json_schema", strict=True)

# Evaluator function
async def final_answer_correct(inputs: dict, outputs: dict, reference_outputs: dict) -> bool:
    """Evaluate if the final response is equivalent to reference response."""
    # Note that we assume the outputs has a 'response' dictionary. We'll need to make sure
    # that the target function we define includes this key.
    user = f"""QUESTION: {inputs['question']}
    GROUND TRUTH RESPONSE: {reference_outputs['response']}
    STUDENT RESPONSE: {outputs['response']}"""

    grade = await grader_llm.ainvoke([{"role": "system", "content": grader_instructions}, {"role": "user", "content": user}])
    return grade["is_correct"]

#### 4. Run the Evaluation

In [40]:
# Evaluation job and results
experiment_results = await client.aevaluate(
    run_graph,
    data=dataset_name,
    evaluators=[final_answer_correct, correctness_evaluator],
    experiment_prefix="agent-e2e",
    num_repetitions=1,
    max_concurrency=5,
)

View the evaluation results for experiment: 'agent-e2e-4906244c' at:
https://smith.langchain.com/o/bcad64b4-50f8-4e66-a0be-8dbaf6f6619c/datasets/6ce1f3e1-7932-4eb5-865d-1f750eceb4c6/compare?selectedSessions=3eb10d5b-15e2-4837-ba82-ca85d1426e7a




4it [00:24,  6.00s/it]


### Evaluating a Single Step of the Agent

Agents generally perform multiple actions. While it is useful to evaluate them end-to-end, it can also be useful to evaluate these individual actions, similar to the concept of unit testing in software development. This generally involves evaluating a single step of the agent - the LLM call where it decides what to do.

- Input: Input to a single step 
- Output: Output of that step, which is usually the LLM response

![single-step](../images/single-step.png) 

#### 1. Create a Dataset for this Single Step

In [41]:
examples = [
    {
        "messages": "My customer ID is 1. What's my most recent purchase? and What albums does the catalog have by U2?", 
        "route": 'transfer_to_invoice_information_subagent'
    },
    {
        "messages": "What songs do you have by U2?", 
        "route": 'transfer_to_music_subagent'
    },
    {
        "messages": "My name is Aaron Mitchell. My number associated with my account is +1 (204) 452-6452. I am trying to find the invoice number for my most recent song purchase. Could you help me with it?", 
        "route": 'transfer_to_invoice_information_subagent'
    },
    {
        "messages": "Who recorded Wish You Were Here again? What other albums by them do you have?", 
        "route": 'transfer_to_music_subagent'
    }
]


dataset_name = "Evals: Single-Step"
if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name=dataset_name)
    client.create_examples(
        inputs = [{"messages": ex["messages"]} for ex in examples],
        outputs = [{"route": ex["route"]} for ex in examples],
        dataset_id=dataset.id
    )

#### 2. Define the Application Logic to Evaluate 

We only need to evaluate the supervisor routing step, so let's add a breakpoint right after the supervisor step.

In [42]:
from langchain_core.messages import HumanMessage

async def run_supervisor_routing(inputs: dict):
    result = await multiagent.ainvoke(
        {"messages": [HumanMessage(content=inputs['messages'])]},
        interrupt_before=["music_subagent", "invoice_information_subagent"],
        config={"thread_id": uuid.uuid4(), "user_id" : "10"}
    )
    return {"route": result["messages"][-1].name}

#### 3. Define the Evaluator

In [43]:
def correct(outputs: dict, reference_outputs: dict) -> bool:
    """Check if the agent chose the correct route."""
    return outputs['route'] == reference_outputs["route"]

#### 4. Run the Evaluation

In [44]:
experiment_results = await client.aevaluate(
    run_supervisor_routing,
    data=dataset_name,
    evaluators=[correct],
    experiment_prefix="agent-singlestep",
    max_concurrency=5,
)

View the evaluation results for experiment: 'agent-singlestep-12ef4bfa' at:
https://smith.langchain.com/o/bcad64b4-50f8-4e66-a0be-8dbaf6f6619c/datasets/97e1645b-76ee-4618-8a20-d6c96c9beb61/compare?selectedSessions=1f089108-dbf3-4585-a558-6bbe50bd9095




4it [00:08,  2.10s/it]


### Evaluating the Trajectory of the Agent

Evaluating an agent's trajectory involves evaluating all the steps an agent took. The evaluator here is some function over the steps taken. Examples of evaluators include an exact match for each tool name in the sequence or the number of "incorrect" steps taken.

- Input: User input to the overall agent 
- Output: A list of steps taken.

![trajectory](../images/trajectory.png) 

#### 1. Create a Dataset

In [52]:
# Create a dataset
examples = [
    {
        "question": "My customer ID is 1. What's my most recent purchase? and What albums does the catalog have by U2?",
        "trajectory": ["supervisor","invoice_information_subagent","supervisor","music_subagent","supervisor"],
    },
    {
        "question": "What songs do you have by U2?",
        "trajectory": ["supervisor","music_subagent","supervisor"],
    },
    {
        "question": "My name is Aaron Mitchell. My number associated with my account is +1 (204) 452-6452. I am trying to find the invoice number for my most recent song purchase. Could you help me with it?",
        "trajectory": ["supervisor","invoice_information_subagent","supervisor"],
    },
    {
        "question": "Who recorded Wish You Were Here again? What other albums by them do you have?",
        "trajectory": ["supervisor","music_subagent","supervisor"],
    },
]

dataset_name = "Evals: Trajectory"

if not client.has_dataset(dataset_name=dataset_name):
    dataset = client.create_dataset(dataset_name=dataset_name)
    client.create_examples(
        inputs=[{"question": ex["question"]} for ex in examples],
        outputs=[{"trajectory": ex["trajectory"]} for ex in examples],
        dataset_id=dataset.id
    )

#### 2. Define the Application Logic to Evaluate 

In [53]:
graph = multiagent

async def run_graph(inputs: dict) -> dict:
    """Run graph and track the trajectory it takes along with the final response."""
    trajectory = []
    thread_id = uuid.uuid4()
    configuration = {"thread_id": thread_id, "user_id" : "10"}

    # Run until interrupt 
    async for chunk in graph.astream({"messages": [
            {
                "role": "user",
                "content": inputs['question'],
            }
        ]}, config = configuration, stream_mode="debug"):
        if chunk['type'] == 'task':
            trajectory.append(chunk['payload']['name'])

    # Resume from interrupt
    async for chunk in graph.astream(Command(resume=""), config = configuration, stream_mode="debug"):
        if chunk['type'] == 'task':
            trajectory.append(chunk['payload']['name'])
    return {"trajectory": trajectory}

#### 3. Define the Evaluator

In [54]:
def evaluate_exact_match(outputs: dict, reference_outputs: dict):
    """Evaluate whether the trajectory exactly matches the expected output"""
    return {
        "key": "exact_match", 
        "score": outputs["trajectory"] == reference_outputs["trajectory"]
    }

def evaluate_extra_steps(outputs: dict, reference_outputs: dict) -> dict:
    """Evaluate the number of unmatched steps in the agent's output."""
    i = j = 0
    unmatched_steps = 0

    while i < len(reference_outputs['trajectory']) and j < len(outputs['trajectory']):
        if reference_outputs['trajectory'][i] == outputs['trajectory'][j]:
            i += 1  # Match found, move to the next step in reference trajectory
        else:
            unmatched_steps += 1  # Step is not part of the reference trajectory
        j += 1  # Always move to the next step in outputs trajectory

    # Count remaining unmatched steps in outputs beyond the comparison loop
    unmatched_steps += len(outputs['trajectory']) - j

    return {
        "key": "unmatched_steps",
        "score": unmatched_steps,
    }

#### 4. Run the Evaluation

In [55]:
experiment_results = await client.aevaluate(
    run_graph,
    data=dataset_name,
    evaluators=[evaluate_extra_steps, evaluate_exact_match],
    experiment_prefix="agent-trajectory",
    num_repetitions=1,
    max_concurrency=4,
)

View the evaluation results for experiment: 'agent-trajectory-b9aac17c' at:
https://smith.langchain.com/o/bcad64b4-50f8-4e66-a0be-8dbaf6f6619c/datasets/3661e21d-1328-41b9-964a-d1e5b54c184c/compare?selectedSessions=6c3cb817-e63f-468d-9b84-644ad46c9516




4it [00:36,  9.07s/it]
